In [1]:
from src.doc2vec import Doc2VecSVM

In [2]:
doc2vecsvm = Doc2VecSVM(doc2vec_model="model_best")

In [4]:
wv = doc2vecsvm.doc2vec_model.wv
wv[]

In [26]:
doc2vecsvm.doc2vec_model.infer_vector(['amazing'])

array([-0.00329152, -0.04856259, -0.01947715, -0.03269903,  0.01845981,
       -0.04572375, -0.01164977, -0.02608692, -0.00625388,  0.03429485,
        0.03114964, -0.03364512, -0.00742597, -0.03584009,  0.07260682,
        0.02262136, -0.04848115, -0.0068965 ,  0.01059397,  0.05005822,
       -0.02359922,  0.01217214, -0.04280048,  0.00933738,  0.04571541,
        0.01816959, -0.01684867,  0.02562549, -0.0022227 ,  0.01787252,
       -0.00547092,  0.01311139, -0.04651301,  0.00769798, -0.00421882,
       -0.09174376, -0.04386034, -0.1724786 ,  0.00383178, -0.07338027,
        0.02013617, -0.00160977,  0.00506224,  0.04001335,  0.03268605,
        0.02001405, -0.00425337,  0.00064853, -0.06148577, -0.05211562,
        0.03144492, -0.01167327, -0.00946847, -0.04851775,  0.0893448 ,
       -0.01202435, -0.03995702, -0.06620809, -0.03483141,  0.02371029,
       -0.01005711,  0.00807613,  0.02910807,  0.01598745, -0.02902009,
       -0.0146014 ,  0.0177265 ,  0.05359872, -0.07474534, -0.03